## Optimizing Tinybert fine-tuned with IMDB dataset using PSO and pruning using L1 pruning 

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Step 1: Load the IMDb dataset
dataset = load_dataset("imdb")

# Step 2: Load TinyBERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")
model = BertForSequenceClassification.from_pretrained("huawei-noah/TinyBERT_General_4L_312D", num_labels=2)
model.to(device)

# Step 3: Preprocess the data
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding=True, max_length=128)

encoded_dataset = dataset.map(preprocess_function, batched=True)

# Step 4: Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,  # L2 regularization
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
)

# Step 5: Implement custom Trainer with L1 regularization
class L1Trainer(Trainer):
    def __init__(self, l1_lambda=1e-5, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.l1_lambda = l1_lambda

    # Override the compute_loss method to add L1 regularization
    def compute_loss(self, model, inputs, return_outputs=False):
        # Compute the standard loss using Hugging Face's method
        loss, outputs = super().compute_loss(model, inputs, return_outputs=True)

        # Apply L1 regularization (penalty for non-zero weights)
        l1_loss = 0
        for param in model.parameters():
            l1_loss += torch.sum(torch.abs(param))

        # Add L1 regularization to the loss
        loss += self.l1_lambda * l1_loss

        return (loss, outputs) if return_outputs else loss

# Step 6: Define the L1Trainer with L1 regularization
trainer = L1Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['test'],
    l1_lambda=1e-5,  # L1 regularization coefficient
)

# Step 7: Train the model with L1 regularization
trainer.train()

# Step 8: Evaluate the model
eval_result = trainer.evaluate()
print(f"Evaluation results: {eval_result}")

# Step 9: Load the accuracy metric for evaluation
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

# Step 10: Define a custom compute_metrics function to calculate accuracy
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Step 11: Redefine the Trainer to include compute_metrics and L1 regularization
trainer = L1Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['test'],
    compute_metrics=compute_metrics,
    l1_lambda=1e-5  # L1 regularization coefficient
)

# Step 12: Evaluate the model with L1 regularization
eval_result = trainer.evaluate()

# Step 13: Print the evaluation results and accuracy
print(f"Evaluation results: {eval_result}")
print(f"Accuracy: {eval_result['eval_accuracy']:.4f}")

In [3]:
model.save_pretrained("./optimized_tinybert")
tokenizer.save_pretrained("./optimized_tinybert")

('./optimized_tinybert\\tokenizer_config.json',
 './optimized_tinybert\\special_tokens_map.json',
 './optimized_tinybert\\vocab.txt',
 './optimized_tinybert\\added_tokens.json')

In [4]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 312, padding_idx=0)
      (position_embeddings): Embedding(512, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-3): 4 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-1

In [5]:
tokenizer

BertTokenizer(name_or_path='huawei-noah/TinyBERT_General_4L_312D', vocab_size=30522, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [1]:
import socket
import torch
import pickle
from transformers import BertForSequenceClassification, BertTokenizer
import torch.nn as nn
from sklearn.metrics import accuracy_score

# Server setup
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host = '0.0.0.0'
port = 10300
server_socket.bind((host, port))
server_socket.listen(1)
print(f"Server listening on {host}:{port}...")

# Load the original TinyBERT model
tinybert = BertForSequenceClassification.from_pretrained("./optimized_tinybert")

class TinyBERTPart1(nn.Module):
    def __init__(self, original_model):
        super(TinyBERTPart1, self).__init__()
        self.encoder_layers = nn.ModuleList(original_model.bert.encoder.layer[:2])  # Access encoder layers

    def forward(self, x):
        for layer in self.encoder_layers:
            x = layer(x)
        return x

class TinyBERTPart2(nn.Module):
    def __init__(self, original_model):
        super(TinyBERTPart2, self).__init__()
        self.encoder_layers = nn.ModuleList(original_model.bert.encoder.layer[2:])  # Access remaining layers
        self.pooler = original_model.bert.pooler  # Access the pooler
        self.classifier = nn.Linear(original_model.config.hidden_size, 2)  # Adjust the output size as needed

    def forward(self, x):
        for layer in self.encoder_layers:
            x = layer(x)
        x = self.pooler(x)  # Use the pooler after the encoder layers
        x = self.classifier(x)  # Classification layer
        return x

model_part1 = TinyBERTPart1(tinybert)
model_part2 = TinyBERTPart2(tinybert)

# Accept connection
client_socket, client_address = server_socket.accept()
print(f"Connected to client at {client_address}")

# Receive the intermediate tensor from client
data = b''
while True:
    packet = client_socket.recv(4096)
    if not packet:
        break
    data += packet

print("Intermediate output received from client.")

# Deserialize the received data
intermediate_output = pickle.loads(data)
print("Deserialized the received data.")

# Perform forward pass through Part 2
with torch.no_grad():
    output = model_part2(intermediate_output)

print(f"Model Part 2 forward pass completed. Output shape: {output.shape}")

# Step 1: Convert the output to logits
predictions = torch.argmax(output, dim=1)  # Get predicted class indices

# Step 2: For evaluation, assume you have a list of true labels
true_labels = torch.tensor([1, 0, 1, 0, 1])  # Example true labels

# Calculate accuracy
accuracy = accuracy_score(true_labels.numpy(), predictions.numpy())
print(f"Accuracy: {accuracy * 100:.2f}%")

# Close the socket connection
client_socket.close()
server_socket.close()
print("Connection closed.")

C:\Users\Exam\.conda\envs\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Server listening on 0.0.0.0:10300...
Connected to client at ('172.16.19.49', 56963)
Intermediate output received from client.
Deserialized the received data.


AttributeError: 'tuple' object has no attribute 'size'

In [2]:
print(f"Type of intermediate_output: {type(intermediate_output)}")


Type of intermediate_output: <class 'torch.Tensor'>


In [5]:
print(f"Shape of intermediate_output: {intermediate_output.shape}")

Shape of intermediate_output: torch.Size([5, 13, 312])


In [6]:
import socket
import torch
import pickle
from transformers import BertForSequenceClassification
import torch.nn as nn
from sklearn.metrics import accuracy_score

# Server setup
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host = '0.0.0.0'
port = 10300
server_socket.bind((host, port))
server_socket.listen(1)
print(f"Server listening on {host}:{port}...")

# Load the original TinyBERT model
tinybert = BertForSequenceClassification.from_pretrained("./optimized_tinybert")

class TinyBERTPart1(nn.Module):
    def __init__(self, original_model):
        super(TinyBERTPart1, self).__init__()
        self.encoder_layers = nn.ModuleList(original_model.bert.encoder.layer[:2])  # Access encoder layers

    def forward(self, x):
        for layer in self.encoder_layers:
            x = layer(x)
        return x

class TinyBERTPart2(nn.Module):
    def __init__(self, original_model):
        super(TinyBERTPart2, self).__init__()
        self.encoder_layers = nn.ModuleList(original_model.bert.encoder.layer[2:])  # Access remaining layers
        self.pooler = original_model.bert.pooler  # Access the pooler
        self.classifier = nn.Linear(original_model.config.hidden_size, 2)  # Adjust the output size as needed

    def forward(self, hidden_states):
        for layer in self.encoder_layers:
            hidden_states = layer(hidden_states)  # Process the input tensor through the layers
        pooled_output = self.pooler(hidden_states)  # Use the pooler after the encoder layers
        logits = self.classifier(pooled_output)  # Classification layer
        return logits

model_part1 = TinyBERTPart1(tinybert)
model_part2 = TinyBERTPart2(tinybert)

# Accept connection
client_socket, client_address = server_socket.accept()
print(f"Connected to client at {client_address}")

# Receive the intermediate tensor from client
data = b''
while True:
    packet = client_socket.recv(4096)
    if not packet:
        break
    data += packet

print("Intermediate output received from client.")

# Deserialize the received data
intermediate_output = pickle.loads(data)
print("Deserialized the received data.")

# Check the shape of intermediate_output
print(f"Type of intermediate_output: {type(intermediate_output)}")
print(f"Shape of intermediate_output: {intermediate_output.shape}")

# Perform forward pass through Part 2
with torch.no_grad():
    print(f"Before Part 2, shape of input: {intermediate_output.shape}")
    output = model_part2(intermediate_output)
    print(f"Output shape from Part 2: {output.shape}")

# Step 1: Convert the output to logits
predictions = torch.argmax(output, dim=1)  # Get predicted class indices

# Step 2: For evaluation, assume you have a list of true labels
true_labels = torch.tensor([1, 0, 1, 0, 1])  # Example true labels

# Calculate accuracy
accuracy = accuracy_score(true_labels.numpy(), predictions.numpy())
print(f"Accuracy: {accuracy * 100:.2f}%")

# Close the socket connection
client_socket.close()
server_socket.close()
print("Connection closed.")


Server listening on 0.0.0.0:10300...
Connected to client at ('172.16.19.49', 56978)
Intermediate output received from client.
Deserialized the received data.
Type of intermediate_output: <class 'torch.Tensor'>
Shape of intermediate_output: torch.Size([5, 13, 312])
Before Part 2, shape of input: torch.Size([5, 13, 312])


AttributeError: 'tuple' object has no attribute 'size'

In [8]:
import socket
import torch
import pickle
from transformers import BertForSequenceClassification
import torch.nn as nn
from sklearn.metrics import accuracy_score

# Server setup
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host = '0.0.0.0'
port = 10300
server_socket.bind((host, port))
server_socket.listen(1)
print(f"Server listening on {host}:{port}...")

# Load the original TinyBERT model
tinybert = BertForSequenceClassification.from_pretrained("./optimized_tinybert")

class TinyBERTPart1(nn.Module):
    def __init__(self, original_model):
        super(TinyBERTPart1, self).__init__()
        self.encoder_layers = nn.ModuleList(original_model.bert.encoder.layer[:2])  # Access encoder layers

    def forward(self, x):
        for layer in self.encoder_layers:
            x = layer(x)[0]  # Get the output tensor from each layer
        return x

class TinyBERTPart2(nn.Module):
    def __init__(self, original_model):
        super(TinyBERTPart2, self).__init__()
        self.encoder_layers = nn.ModuleList(original_model.bert.encoder.layer[2:])  # Access remaining layers
        self.pooler = original_model.bert.pooler  # Access the pooler
        self.classifier = nn.Linear(original_model.config.hidden_size, 2)  # Adjust the output size as needed

    def forward(self, hidden_states):
        for layer in self.encoder_layers:
            output = layer(hidden_states)
            hidden_states = output[0] if isinstance(output, tuple) else output  # Unpack if it's a tuple
        pooled_output = self.pooler(hidden_states)  # Use the pooler after the encoder layers
        logits = self.classifier(pooled_output)  # Classification layer
        return logits

model_part1 = TinyBERTPart1(tinybert)
model_part2 = TinyBERTPart2(tinybert)

# Accept connection
client_socket, client_address = server_socket.accept()
print(f"Connected to client at {client_address}")

# Receive the intermediate tensor from client
data = b''
while True:
    packet = client_socket.recv(4096)
    if not packet:
        break
    data += packet

print("Intermediate output received from client.")

# Deserialize the received data
intermediate_output = pickle.loads(data)
print("Deserialized the received data.")

# Check the shape of intermediate_output
print(f"Type of intermediate_output: {type(intermediate_output)}")
print(f"Shape of intermediate_output: {intermediate_output.shape}")

# Perform forward pass through Part 2
with torch.no_grad():
    print(f"Before Part 2, shape of input: {intermediate_output.shape}")
    output = model_part2(intermediate_output)
    print(f"Output shape from Part 2: {output.shape}")

# Step 1: Convert the output to logits
predictions = torch.argmax(output, dim=1)  # Get predicted class indices

# Step 2: For evaluation, assume you have a list of true labels
true_labels = torch.tensor([1, 0, 1, 0, 1])  # Example true labels

# Calculate accuracy
accuracy = accuracy_score(true_labels.numpy(), predictions.numpy())
print(f"Accuracy: {accuracy * 100:.2f}%")

# Close the socket connection
client_socket.close()
server_socket.close()
print("Connection closed.")


Server listening on 0.0.0.0:10300...
Connected to client at ('172.16.19.49', 56997)
Intermediate output received from client.
Deserialized the received data.
Type of intermediate_output: <class 'torch.Tensor'>
Shape of intermediate_output: torch.Size([5, 13, 312])
Before Part 2, shape of input: torch.Size([5, 13, 312])
Output shape from Part 2: torch.Size([5, 2])
Accuracy: 60.00%
Connection closed.


In [10]:
model_part2

TinyBERTPart2(
  (encoder_layers): ModuleList(
    (0-1): 2 x BertLayer(
      (attention): BertAttention(
        (self): BertSdpaSelfAttention(
          (query): Linear(in_features=312, out_features=312, bias=True)
          (key): Linear(in_features=312, out_features=312, bias=True)
          (value): Linear(in_features=312, out_features=312, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (output): BertSelfOutput(
          (dense): Linear(in_features=312, out_features=312, bias=True)
          (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (intermediate): BertIntermediate(
        (dense): Linear(in_features=312, out_features=1200, bias=True)
        (intermediate_act_fn): GELUActivation()
      )
      (output): BertOutput(
        (dense): Linear(in_features=1200, out_features=312, bias=True)
        (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise

In [12]:
import socket
import torch
import pickle
from transformers import BertForSequenceClassification
import torch.nn as nn
from sklearn.metrics import accuracy_score

# Server setup
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host = '0.0.0.0'
port = 10300
server_socket.bind((host, port))
server_socket.listen(1)
print(f"Server listening on {host}:{port}...")

# Load the original TinyBERT model
tinybert = BertForSequenceClassification.from_pretrained("./optimized_tinybert")

class TinyBERTPart2(nn.Module):
    def __init__(self, original_model):
        super(TinyBERTPart2, self).__init__()
        self.encoder_layers = nn.ModuleList(original_model.bert.encoder.layer[2:])  # Access remaining layers
        self.pooler = original_model.bert.pooler  # Access the pooler
        self.classifier = nn.Linear(original_model.config.hidden_size, 2)  # Adjust the output size as needed

    def forward(self, hidden_states):
        for layer in self.encoder_layers:
            output = layer(hidden_states)
            hidden_states = output[0] if isinstance(output, tuple) else output  # Unpack if it's a tuple
        pooled_output = self.pooler(hidden_states)  # Use the pooler after the encoder layers
        logits = self.classifier(pooled_output)  # Classification layer
        return logits

model_part2 = TinyBERTPart2(tinybert)

# Accept connection
client_socket, client_address = server_socket.accept()
print(f"Connected to client at {client_address}")

# Receive the intermediate tensor from client
data = b''
while True:
    packet = client_socket.recv(4096)
    if not packet:
        break
    data += packet

print("Intermediate output received from client.")

# Deserialize the received data
intermediate_output = pickle.loads(data)
print("Deserialized the received data.")

# Check the shape of intermediate_output
print(f"Type of intermediate_output: {type(intermediate_output)}")
print(f"Shape of intermediate_output: {intermediate_output.shape}")

# Perform forward pass through Part 2
with torch.no_grad():
    print(f"Before Part 2, shape of input: {intermediate_output.shape}")
    output = model_part2(intermediate_output)
    print(f"Output shape from Part 2: {output.shape}")

# Step 1: Convert the output to logits
predictions = torch.argmax(output, dim=1)  # Get predicted class indices

# Step 2: For evaluation, assume you have a list of true labels
true_labels = torch.tensor([1, 0, 1, 0, 1])  # Example true labels

# Calculate accuracy
accuracy = accuracy_score(true_labels.numpy(), predictions.numpy())
print(f"Accuracy: {accuracy * 100:.2f}%")

# Close the socket connection
client_socket.close()
server_socket.close()
print("Connection closed.")


Server listening on 0.0.0.0:10300...
Connected to client at ('172.16.19.49', 57013)
Intermediate output received from client.
Deserialized the received data.
Type of intermediate_output: <class 'torch.Tensor'>
Shape of intermediate_output: torch.Size([5, 13, 312])
Before Part 2, shape of input: torch.Size([5, 13, 312])
Output shape from Part 2: torch.Size([5, 2])
Accuracy: 40.00%
Connection closed.


In [13]:
model_part1

TinyBERTPart1(
  (encoder_layers): ModuleList(
    (0-1): 2 x BertLayer(
      (attention): BertAttention(
        (self): BertSdpaSelfAttention(
          (query): Linear(in_features=312, out_features=312, bias=True)
          (key): Linear(in_features=312, out_features=312, bias=True)
          (value): Linear(in_features=312, out_features=312, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (output): BertSelfOutput(
          (dense): Linear(in_features=312, out_features=312, bias=True)
          (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (intermediate): BertIntermediate(
        (dense): Linear(in_features=312, out_features=1200, bias=True)
        (intermediate_act_fn): GELUActivation()
      )
      (output): BertOutput(
        (dense): Linear(in_features=1200, out_features=312, bias=True)
        (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise

In [20]:
import socket
import torch
import pickle
from transformers import BertForSequenceClassification
import torch.nn as nn
from sklearn.metrics import accuracy_score

# Server setup
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host = '0.0.0.0'
port = 10300
server_socket.bind((host, port))
server_socket.listen(1)
print(f"Server listening on {host}:{port}...")

# Load the original TinyBERT model
tinybert = BertForSequenceClassification.from_pretrained("./optimized_tinybert")

# Access remaining encoder layers for Part 2
model_part2_layers = tinybert.bert.encoder.layer[2:]  # Access remaining layers
modelpart2 = nn.ModuleList(model_part2_layers)  # Wrap layers in ModuleList

# Accept connection
client_socket, client_address = server_socket.accept()
print(f"Connected to client at {client_address}")

# Receive the intermediate tensor from client
data = b''
while True:
    packet = client_socket.recv(4096)
    if not packet:
        break
    data += packet

print("Intermediate output received from client.")

# Deserialize the received data
intermediate_output = pickle.loads(data)
print("Deserialized the received data.")

# Check the shape of intermediate_output
print(f"Type of intermediate_output: {type(intermediate_output)}")
print(f"Shape of intermediate_output: {intermediate_output.shape}")

# Perform forward pass through Part 2
with torch.no_grad():
    print(f"Before Part 2, shape of input: {intermediate_output.shape}")
    
    # Initialize hidden_states with intermediate_output
    hidden_states = intermediate_output
    
    # Forward through each layer in Part 2
    for layer in modelpart2:
        hidden_states = layer(hidden_states)[0]  # Get the output from each layer

    # Pooling and final classification (you may need to adapt this part)
    pooled_output = tinybert.bert.pooler(hidden_states)
    output = tinybert.classifier(pooled_output)

    print(f"Output shape from Part 2: {output.shape}")

# Step 1: Convert the output to logits
predictions = torch.argmax(output, dim=1)  # Get predicted class indices

# Step 2: For evaluation, assume you have a list of true labels
true_labels = torch.tensor([1, 0, 1, 0, 1])  # Example true labels

# Calculate accuracy
accuracy = accuracy_score(true_labels.numpy(), predictions.numpy())
print(f"Accuracy: {accuracy * 100:.2f}%")

# Close the socket connection
client_socket.close()
server_socket.close()
print("Connection closed.")


Server listening on 0.0.0.0:10300...
Connected to client at ('172.16.19.49', 57088)
Intermediate output received from client.
Deserialized the received data.
Type of intermediate_output: <class 'torch.Tensor'>
Shape of intermediate_output: torch.Size([5, 13, 312])
Before Part 2, shape of input: torch.Size([5, 13, 312])
Output shape from Part 2: torch.Size([5, 2])
Accuracy: 60.00%
Connection closed.


In [21]:
modelpart2

ModuleList(
  (0-1): 2 x BertLayer(
    (attention): BertAttention(
      (self): BertSdpaSelfAttention(
        (query): Linear(in_features=312, out_features=312, bias=True)
        (key): Linear(in_features=312, out_features=312, bias=True)
        (value): Linear(in_features=312, out_features=312, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (output): BertSelfOutput(
        (dense): Linear(in_features=312, out_features=312, bias=True)
        (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (intermediate): BertIntermediate(
      (dense): Linear(in_features=312, out_features=1200, bias=True)
      (intermediate_act_fn): GELUActivation()
    )
    (output): BertOutput(
      (dense): Linear(in_features=1200, out_features=312, bias=True)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
)

In [22]:
model_part2_layers

ModuleList(
  (0-1): 2 x BertLayer(
    (attention): BertAttention(
      (self): BertSdpaSelfAttention(
        (query): Linear(in_features=312, out_features=312, bias=True)
        (key): Linear(in_features=312, out_features=312, bias=True)
        (value): Linear(in_features=312, out_features=312, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (output): BertSelfOutput(
        (dense): Linear(in_features=312, out_features=312, bias=True)
        (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (intermediate): BertIntermediate(
      (dense): Linear(in_features=312, out_features=1200, bias=True)
      (intermediate_act_fn): GELUActivation()
    )
    (output): BertOutput(
      (dense): Linear(in_features=1200, out_features=312, bias=True)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
)

In [23]:
from datasets import load_dataset
dataset = load_dataset("imdb")

In [27]:
print(dataset.column_names)

{'train': ['text', 'label'], 'test': ['text', 'label'], 'unsupervised': ['text', 'label']}


In [28]:
print(dataset['test'])

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})


In [29]:
from datasets import load_dataset

# Load the IMDb dataset
dataset = load_dataset("imdb")

# Access the test dataset
test_data = dataset['test']

# Display the first 10 test samples
for i in range(10):
    print(f"Sample {i+1}:")
    print(test_data[i])
    print()


Sample 1:
{'text': 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s r

In [24]:
import socket
import torch
import pickle
from transformers import BertForSequenceClassification
import torch.nn as nn
from sklearn.metrics import accuracy_score

# Server setup
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host = '0.0.0.0'
port = 10300
server_socket.bind((host, port))
server_socket.listen(1)
print(f"Server listening on {host}:{port}...")

# Load the original TinyBERT model
tinybert = BertForSequenceClassification.from_pretrained("./PSO+Pruning_model/")

# Access remaining encoder layers for Part 2
model_part2_layers = tinybert.bert.encoder.layer[2:]  # Access remaining layers
modelpart2 = nn.ModuleList(model_part2_layers)  # Wrap layers in ModuleList

# Accept connection
client_socket, client_address = server_socket.accept()
print(f"Connected to client at {client_address}")

# Receive the intermediate tensor from client
data = b''
while True:
    packet = client_socket.recv(4096)
    if not packet:
        break
    data += packet

print("Intermediate output received from client.")

# Deserialize the received data
intermediate_output = pickle.loads(data)
print("Deserialized the received data.")

# Check the shape of intermediate_output
print(f"Type of intermediate_output: {type(intermediate_output)}")
print(f"Shape of intermediate_output: {intermediate_output.shape}")

# Perform forward pass through Part 2
with torch.no_grad():
    print(f"Before Part 2, shape of input: {intermediate_output.shape}")
    
    # Initialize hidden_states with intermediate_output
    hidden_states = intermediate_output
    
    # Forward through each layer in Part 2
    for layer in modelpart2:
        hidden_states = layer(hidden_states)[0]  # Get the output from each layer

    # Pooling and final classification (you may need to adapt this part)
    pooled_output = tinybert.bert.pooler(hidden_states)
    output = tinybert.classifier(pooled_output)

    print(f"Output shape from Part 2: {output.shape}")

# Step 1: Convert the output to logits
predictions = torch.argmax(output, dim=1)  # Get predicted class indices

# Step 2: For evaluation, assume you have a list of true labels
true_labels = torch.tensor([1, 0, 1, 0, 1])  # Example true labels

# Calculate accuracy
accuracy = accuracy_score(true_labels.numpy(), predictions.numpy())
print(f"Accuracy: {accuracy * 100:.2f}%")

# Close the socket connection
client_socket.close()
server_socket.close()
print("Connection closed.")


Server listening on 0.0.0.0:10300...
Connected to client at ('172.16.16.77', 52488)
Intermediate output received from client.
Deserialized the received data.
Type of intermediate_output: <class 'torch.Tensor'>
Shape of intermediate_output: torch.Size([5, 13, 128])
Before Part 2, shape of input: torch.Size([5, 13, 128])
Output shape from Part 2: torch.Size([5, 2])
Accuracy: 80.00%
Connection closed.


In [25]:
predictions

tensor([1, 0, 1, 1, 1])

In [26]:
true_labels

tensor([1, 0, 1, 0, 1])

In [27]:
modelpart2

ModuleList()

In [28]:
model_part2_layers

ModuleList()

In [29]:
tinybert

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

In [30]:
 tinybert.bert.encoder.layer[0]

BertLayer(
  (attention): BertAttention(
    (self): BertSdpaSelfAttention(
      (query): Linear(in_features=128, out_features=128, bias=True)
      (key): Linear(in_features=128, out_features=128, bias=True)
      (value): Linear(in_features=128, out_features=128, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (output): BertSelfOutput(
      (dense): Linear(in_features=128, out_features=128, bias=True)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (intermediate): BertIntermediate(
    (dense): Linear(in_features=128, out_features=512, bias=True)
    (intermediate_act_fn): GELUActivation()
  )
  (output): BertOutput(
    (dense): Linear(in_features=512, out_features=128, bias=True)
    (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)

In [31]:
 tinybert.bert.encoder.layer[1]

BertLayer(
  (attention): BertAttention(
    (self): BertSdpaSelfAttention(
      (query): Linear(in_features=128, out_features=128, bias=True)
      (key): Linear(in_features=128, out_features=128, bias=True)
      (value): Linear(in_features=128, out_features=128, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (output): BertSelfOutput(
      (dense): Linear(in_features=128, out_features=128, bias=True)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (intermediate): BertIntermediate(
    (dense): Linear(in_features=128, out_features=512, bias=True)
    (intermediate_act_fn): GELUActivation()
  )
  (output): BertOutput(
    (dense): Linear(in_features=512, out_features=128, bias=True)
    (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)

In [32]:
tinybert

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

In [39]:
import socket
import torch
import pickle
from transformers import BertForSequenceClassification
from sklearn.metrics import accuracy_score

# Server setup
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host = '0.0.0.0'
port = 10300
server_socket.bind((host, port))
server_socket.listen(1)
print(f"Server listening on {host}:{port}...")

# Load TinyBERT model
tinybert = BertForSequenceClassification.from_pretrained("./PSO+Pruning_model/")

# Accept client connection
client_socket, client_address = server_socket.accept()
print(f"Connected to client at {client_address}")

# Receive the intermediate tensor from client
data = b''
while True:
    packet = client_socket.recv(4096)
    if not packet:
        break
    data += packet

print("Intermediate output received from client.")

# Deserialize received data
intermediate_output = pickle.loads(data)
print("Deserialized the received data.")

# Continue forward pass through the remaining layers
with torch.no_grad():
    # Pass through the second BertLayer (Layer 1)
    hidden_states = tinybert.bert.encoder.layer[1](intermediate_output)[0]

    # Pooling and classification layers
    pooled_output = tinybert.bert.pooler(hidden_states)
    output = tinybert.classifier(pooled_output)

# Convert output to logits and get predictions
predictions = torch.argmax(output, dim=1)

# Define true labels for accuracy calculation (example labels)
true_labels = torch.tensor([1, 0, 1, 0, 1])

# Calculate accuracy
accuracy = accuracy_score(true_labels.numpy(), predictions.numpy())
print(f"Accuracy: {accuracy * 100:.2f}%")

# Close socket connection
client_socket.close()
server_socket.close()
print("Connection closed.")


Server listening on 0.0.0.0:10300...
Connected to client at ('172.16.16.77', 52785)
Intermediate output received from client.
Deserialized the received data.
Accuracy: 80.00%
Connection closed.


In [43]:
tinybert.bert.encoder.layer[1]

BertLayer(
  (attention): BertAttention(
    (self): BertSdpaSelfAttention(
      (query): Linear(in_features=128, out_features=128, bias=True)
      (key): Linear(in_features=128, out_features=128, bias=True)
      (value): Linear(in_features=128, out_features=128, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (output): BertSelfOutput(
      (dense): Linear(in_features=128, out_features=128, bias=True)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (intermediate): BertIntermediate(
    (dense): Linear(in_features=128, out_features=512, bias=True)
    (intermediate_act_fn): GELUActivation()
  )
  (output): BertOutput(
    (dense): Linear(in_features=512, out_features=128, bias=True)
    (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)

In [44]:
# Server Side Code (modelpart2)
import socket
import torch
import pickle
from transformers import BertForSequenceClassification
import torch.nn as nn
from sklearn.metrics import accuracy_score

# Server setup
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host = '0.0.0.0'
port = 10300
server_socket.bind((host, port))
server_socket.listen(1)
print(f"Server listening on {host}:{port}...")

# Load TinyBERT model
tinybert = BertForSequenceClassification.from_pretrained("./PSO+Pruning_model/")

# Define modelpart2 (remaining encoder layers, pooler, and classifier)
modelpart2_layers = tinybert.bert.encoder.layer[1:]  # Remaining encoder layers
modelpart2 = nn.ModuleList(modelpart2_layers)  # Wrap remaining layers in ModuleList

# Accept connection from client
client_socket, client_address = server_socket.accept()
print(f"Connected to client at {client_address}")

# Receive the intermediate tensor from client
data = b''
while True:
    packet = client_socket.recv(4096)
    if not packet:
        break
    data += packet

print("Intermediate output received from client.")

# Deserialize the received data
intermediate_output = pickle.loads(data)

# Pass through modelpart2
with torch.no_grad():
    hidden_states = intermediate_output  # Start from intermediate output received

    # Forward pass through each layer in modelpart2
    for layer in modelpart2:
        hidden_states = layer(hidden_states)[0]

    # Pooling and final classification
    pooled_output = tinybert.bert.pooler(hidden_states)
    output = tinybert.classifier(pooled_output)

# Get predictions and calculate accuracy (using example true labels)
predictions = torch.argmax(output, dim=1)  # Predicted class indices
true_labels = torch.tensor([1, 0, 1, 0, 1])  # Example true labels

accuracy = accuracy_score(true_labels.numpy(), predictions.numpy())
print(f"Accuracy: {accuracy * 100:.2f}%")

# Close the socket connection
client_socket.close()
server_socket.close()
print("Connection closed.")


Server listening on 0.0.0.0:10300...
Connected to client at ('172.16.16.77', 52852)
Intermediate output received from client.
Accuracy: 80.00%
Connection closed.


In [ ]:
# Define modelpart2 to include remaining encoder layers, pooler, and classifier
modelpart2_layers = nn.ModuleList([
    *tinybert.bert.encoder.layer[1:],   # Remaining encoder layers after the first
    tinybert.bert.pooler                # Pooling layer
])
modelpart2_classifier = tinybert.classifier  # Classifier layer

# Forward pass through modelpart2
with torch.no_grad():
    hidden_states = intermediate_output  # Start from the intermediate output received

    # Pass through each encoder layer in modelpart2
    for layer in modelpart2_layers:
        hidden_states = layer(hidden_states)[0]

    # Pooling and final classification
    pooled_output = modelpart2_layers[-1](hidden_states)  # Pooling layer output
    output = modelpart2_classifier(pooled_output)  # Classification layer output


In [ ]:
# Server Side Code (modelpart2)
import socket
import torch
import pickle
from transformers import BertForSequenceClassification
import torch.nn as nn
from sklearn.metrics import accuracy_score

# Server setup
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host = '0.0.0.0'
port = 10300
server_socket.bind((host, port))
server_socket.listen(1)
print(f"Server listening on {host}:{port}...")

# Load TinyBERT model
tinybert = BertForSequenceClassification.from_pretrained("./PSO+Pruning_model/")

# Define modelpart2 (remaining encoder layers, pooler, and classifier)
modelpart2_layers = tinybert.bert.encoder.layer[1:]  # Remaining encoder layers
modelpart2 = nn.ModuleList(modelpart2_layers)  # Wrap remaining layers in ModuleList

# Accept connection from client
client_socket, client_address = server_socket.accept()
print(f"Connected to client at {client_address}")

# Receive the intermediate tensor from client
data = b''
while True:
    packet = client_socket.recv(4096)
    if not packet:
        break
    data += packet

print("Intermediate output received from client.")

# Deserialize the received data
intermediate_output = pickle.loads(data)

# Pass through modelpart2
with torch.no_grad():
    hidden_states = intermediate_output  # Start from intermediate output received

    # Forward pass through each layer in modelpart2
    for layer in modelpart2:
        hidden_states = layer(hidden_states)[0]

    # Pooling and final classification
    pooled_output = tinybert.bert.pooler(hidden_states)
    output = tinybert.classifier(pooled_output)

# Get predictions and calculate accuracy (using example true labels)
predictions = torch.argmax(output, dim=1)  # Predicted class indices
true_labels = torch.tensor([1, 0, 1, 0, 1])  # Example true labels

accuracy = accuracy_score(true_labels.numpy(), predictions.numpy())
print(f"Accuracy: {accuracy * 100:.2f}%")

# Close the socket connection
client_socket.close()
server_socket.close()
print("Connection closed.")


In [53]:
# Server Side Code (modelpart2)
import socket
import torch
import pickle
from transformers import BertForSequenceClassification
import torch.nn as nn
from sklearn.metrics import accuracy_score

# Server setup
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host = '0.0.0.0'
port = 10300
server_socket.bind((host, port))
server_socket.listen(1)
print(f"Server listening on {host}:{port}...")

# Load TinyBERT model
tinybert = BertForSequenceClassification.from_pretrained("./PSO+Pruning_model/")

# Define modelpart2 to include remaining encoder layers, pooler, and classifier
modelpart2_layers = nn.ModuleList([
    *tinybert.bert.encoder.layer[1:],  # Remaining encoder layers after the first
    tinybert.bert.pooler                # Pooling layer
])

# Combine modelpart2_layers and classifier into a single module
class ModelPart2(nn.Module):
    def __init__(self, layers, classifier):
        super(ModelPart2, self).__init__()
        self.layers = layers
        self.classifier = classifier

    def forward(self, hidden_states):
        # Pass through each encoder layer in modelpart2
        for layer in self.layers[:-1]:  # Exclude the pooling layer for now
            hidden_states = layer(hidden_states)[0]
        
        # Now, use the last layer's output for pooling
        pooled_output = self.layers[-1](hidden_states)  # Pooling layer output
        
        # Classification
        output = self.classifier(pooled_output)  # Classification layer output
        return output

# Instantiate modelpart2
modelpart2 = ModelPart2(modelpart2_layers, tinybert.classifier)

# Accept connection from client
client_socket, client_address = server_socket.accept()
print(f"Connected to client at {client_address}")

# Receive the intermediate tensor from client
data = b''
while True:
    packet = client_socket.recv(4096)
    if not packet:
        break
    data += packet

print("Intermediate output received from client.")

# Deserialize the received data
intermediate_output = pickle.loads(data)

# Check the shape of intermediate_output
print(f"Intermediate output shape: {intermediate_output.shape}")  # Add this line for debugging

# Forward pass through modelpart2
with torch.no_grad():
    hidden_states = intermediate_output  # Start from the intermediate output received
    output = modelpart2(hidden_states)  # Get the final output

# Get predictions and calculate accuracy (using example true labels)
predictions = torch.argmax(output, dim=1)  # Predicted class indices
true_labels = torch.tensor([1, 0, 1, 0, 1])  # Example true labels

accuracy = accuracy_score(true_labels.numpy(), predictions.numpy())
print(f"Accuracy: {accuracy * 100:.2f}%")

# Close the socket connection
client_socket.close()
server_socket.close()
print("Connection closed.")

# Print the model structure
print(modelpart2)


Server listening on 0.0.0.0:10300...
Connected to client at ('172.16.16.77', 53098)
Intermediate output received from client.
Intermediate output shape: torch.Size([5, 13, 128])
Accuracy: 80.00%
Connection closed.
ModelPart2(
  (layers): ModuleList(
    (0): BertLayer(
      (attention): BertAttention(
        (self): BertSdpaSelfAttention(
          (query): Linear(in_features=128, out_features=128, bias=True)
          (key): Linear(in_features=128, out_features=128, bias=True)
          (value): Linear(in_features=128, out_features=128, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (output): BertSelfOutput(
          (dense): Linear(in_features=128, out_features=128, bias=True)
          (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (intermediate): BertIntermediate(
        (dense): Linear(in_features=128, out_features=512, bias=True)
        (intermediat

In [63]:
# Print each layer in modelpart2
for i, layer in enumerate(modelpart2.layers):
    print(f"Layer {i}: {layer}")

# Print the classifier
print(f"Classifier: {modelpart2.classifier}")


Layer 0: BertLayer(
  (attention): BertAttention(
    (self): BertSdpaSelfAttention(
      (query): Linear(in_features=128, out_features=128, bias=True)
      (key): Linear(in_features=128, out_features=128, bias=True)
      (value): Linear(in_features=128, out_features=128, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (output): BertSelfOutput(
      (dense): Linear(in_features=128, out_features=128, bias=True)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (intermediate): BertIntermediate(
    (dense): Linear(in_features=128, out_features=512, bias=True)
    (intermediate_act_fn): GELUActivation()
  )
  (output): BertOutput(
    (dense): Linear(in_features=512, out_features=128, bias=True)
    (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)
Layer 1: BertPooler(
  (dense): Linear(in_features=128, out_features=1

In [64]:
modelpart2

ModelPart2(
  (layers): ModuleList(
    (0): BertLayer(
      (attention): BertAttention(
        (self): BertSdpaSelfAttention(
          (query): Linear(in_features=128, out_features=128, bias=True)
          (key): Linear(in_features=128, out_features=128, bias=True)
          (value): Linear(in_features=128, out_features=128, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (output): BertSelfOutput(
          (dense): Linear(in_features=128, out_features=128, bias=True)
          (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (intermediate): BertIntermediate(
        (dense): Linear(in_features=128, out_features=512, bias=True)
        (intermediate_act_fn): GELUActivation()
      )
      (output): BertOutput(
        (dense): Linear(in_features=512, out_features=128, bias=True)
        (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
     

In [65]:
# Server Side Code (modelpart2)
import socket
import torch
import pickle
from transformers import BertForSequenceClassification
import torch.nn as nn
from sklearn.metrics import accuracy_score

# Server setup
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host = '0.0.0.0'
port = 10300
server_socket.bind((host, port))
server_socket.listen(1)
print(f"Server listening on {host}:{port}...")

# Load TinyBERT model
tinybert = BertForSequenceClassification.from_pretrained("./PSO+Pruning_model/")

# Define modelpart2 to include remaining encoder layers, pooler, and classifier
modelpart2_layers = nn.ModuleList([
    *tinybert.bert.encoder.layer[1:],  # Remaining encoder layers after the first
    tinybert.bert.pooler                # Pooling layer
])

# Combine modelpart2_layers and classifier into a single module
class ModelPart2(nn.Module):
    def __init__(self, layers, classifier):
        super(ModelPart2, self).__init__()
        self.layers = layers
        self.classifier = classifier

    def forward(self, hidden_states):
        # Pass through each encoder layer in modelpart2
        for layer in self.layers[:-1]:  # Exclude the pooling layer for now
            hidden_states = layer(hidden_states)[0]
        
        # Now, use the last layer's output for pooling
        pooled_output = self.layers[-1](hidden_states)  # Pooling layer output
        
        # Classification
        output = self.classifier(pooled_output)  # Classification layer output
        return output

# Instantiate modelpart2
modelpart2 = ModelPart2(modelpart2_layers, tinybert.classifier)

# Accept connection from client
client_socket, client_address = server_socket.accept()
print(f"Connected to client at {client_address}")

# Receive the intermediate tensor from client
data = b''
while True:
    packet = client_socket.recv(4096)
    if not packet:
        break
    data += packet

print("Intermediate output received from client.")

# Deserialize the received data
intermediate_output = pickle.loads(data)

# Check the shape of intermediate_output
print(f"Intermediate output shape: {intermediate_output.shape}")  # Add this line for debugging

# Forward pass through modelpart2
with torch.no_grad():
    hidden_states = intermediate_output  # Start from the intermediate output received
    output = modelpart2(hidden_states)  # Get the final output

# Get predictions and calculate accuracy (using example true labels)
predictions = torch.argmax(output, dim=1)  # Predicted class indices
true_labels = torch.tensor([1, 0, 1, 0, 1])  # Example true labels

accuracy = accuracy_score(true_labels.numpy(), predictions.numpy())
print(f"Accuracy: {accuracy * 100:.2f}%")

# Close the socket connection
client_socket.close()
server_socket.close()
print("Connection closed.")

# Print the model structure
print(modelpart2)


Server listening on 0.0.0.0:10300...
Connected to client at ('172.16.16.77', 53247)
Intermediate output received from client.
Intermediate output shape: torch.Size([5, 13, 128])
Accuracy: 80.00%
Connection closed.
ModelPart2(
  (layers): ModuleList(
    (0): BertLayer(
      (attention): BertAttention(
        (self): BertSdpaSelfAttention(
          (query): Linear(in_features=128, out_features=128, bias=True)
          (key): Linear(in_features=128, out_features=128, bias=True)
          (value): Linear(in_features=128, out_features=128, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (output): BertSelfOutput(
          (dense): Linear(in_features=128, out_features=128, bias=True)
          (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (intermediate): BertIntermediate(
        (dense): Linear(in_features=128, out_features=512, bias=True)
        (intermediat

In [66]:
predictions

tensor([1, 0, 1, 1, 1])

In [67]:
true_labels

tensor([1, 0, 1, 0, 1])

In [2]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [3]:
device

device(type='cuda')

In [ ]:
import socket
import torch
import pickle
from transformers import BertForSequenceClassification
import torch.nn as nn

# Server setup
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host = '0.0.0.0'
port = 10300
server_socket.bind((host, port))
server_socket.listen(1)
print(f"Server listening on {host}:{port}...")

# Load TinyBERT model
tinybert = BertForSequenceClassification.from_pretrained("./PSO+Pruning_model/")

# Define modelpart2 to include remaining encoder layers, pooler, and classifier
modelpart2_layers = nn.ModuleList([
    *tinybert.bert.encoder.layer[1:],  # Remaining encoder layers after the first
    tinybert.bert.pooler                # Pooling layer
])

# Combine modelpart2_layers and classifier into a single module
class ModelPart2(nn.Module):
    def __init__(self, layers, classifier):
        super(ModelPart2, self).__init__()
        self.layers = layers
        self.classifier = classifier

    def forward(self, hidden_states):
        # Pass through each encoder layer in modelpart2
        for layer in self.layers[:-1]:  # Exclude the pooling layer for now
            hidden_states = layer(hidden_states)[0]
        
        # Pooling layer
        pooled_output = self.layers[-1](hidden_states)  # Pooling layer output
        
        # Classification
        output = self.classifier(pooled_output)  # Classification layer output
        return output

# Instantiate modelpart2
modelpart2 = ModelPart2(modelpart2_layers, tinybert.classifier)

# Accept connection from client
client_socket, client_address = server_socket.accept()
print(f"Connected to client at {client_address}")

# Loop to receive multiple inputs and predict until client sends "STOP"
while True:
    # Receive the intermediate tensor from client
    data = b''
    while True:
        packet = client_socket.recv(4096)
        if not packet:
            break
        data += packet
    
    # Check if the client wants to stop
    if data.decode("utf-8") == "STOP":
        print("Received stop signal from client.")
        break

    # Deserialize the received data
    intermediate_output = pickle.loads(data)

    # Check the shape of intermediate_output (optional for debugging)
    print(f"Intermediate output shape: {intermediate_output.shape}")

    # Forward pass through modelpart2
    with torch.no_grad():
        hidden_states = intermediate_output  # Start from the intermediate output received
        output = modelpart2(hidden_states)  # Get the final output

    # Get predictions
    predictions = torch.argmax(output, dim=1)  # Predicted class indices

    # Send predictions back to client
    client_socket.sendall(pickle.dumps(predictions.tolist()))
    print(f"Sent prediction: {predictions.tolist()}")

# Close the socket connection
client_socket.close()
server_socket.close()
print("Connection closed.")


C:\Users\Exam\.conda\envs\Project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Server listening on 0.0.0.0:10300...
Connected to client at ('172.16.16.77', 58308)


In [1]:
# Server Side Code (modelpart2)
import socket
import torch
import pickle
from transformers import BertForSequenceClassification
import torch.nn as nn
from sklearn.metrics import accuracy_score

# Server setup
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host = '0.0.0.0'
port = 10300
server_socket.bind((host, port))
server_socket.listen(1)
print(f"Server listening on {host}:{port}...")

# Load TinyBERT model
tinybert = BertForSequenceClassification.from_pretrained("./PSO+Pruning_model/")

# Define modelpart2 to include remaining encoder layers, pooler, and classifier
modelpart2_layers = nn.ModuleList([
    *tinybert.bert.encoder.layer[1:],  # Remaining encoder layers after the first
    tinybert.bert.pooler                # Pooling layer
])

# Combine modelpart2_layers and classifier into a single module
class ModelPart2(nn.Module):
    def __init__(self, layers, classifier):
        super(ModelPart2, self).__init__()
        self.layers = layers
        self.classifier = classifier

    def forward(self, hidden_states):
        # Pass through each encoder layer in modelpart2
        for layer in self.layers[:-1]:  # Exclude the pooling layer for now
            hidden_states = layer(hidden_states)[0]
        
        # Now, use the last layer's output for pooling
        pooled_output = self.layers[-1](hidden_states)  # Pooling layer output
        
        # Classification
        output = self.classifier(pooled_output)  # Classification layer output
        return output

# Instantiate modelpart2
modelpart2 = ModelPart2(modelpart2_layers, tinybert.classifier)

# Accept connection from client
client_socket, client_address = server_socket.accept()
print(f"Connected to client at {client_address}")

# Receive the intermediate tensor from client
data = b''
while True:
    packet = client_socket.recv(4096)
    if not packet:
        break
    data += packet

print("Intermediate output received from client.")

# Deserialize the received data
intermediate_output = pickle.loads(data)

# Check the shape of intermediate_output
print(f"Intermediate output shape: {intermediate_output.shape}")  # Add this line for debugging

# Forward pass through modelpart2
with torch.no_grad():
    hidden_states = intermediate_output  # Start from the intermediate output received
    output = modelpart2(hidden_states)  # Get the final output

# Get predictions and calculate accuracy (using example true labels)
predictions = torch.argmax(output, dim=1)  # Predicted class indices
true_labels = torch.tensor([1, 0, 1, 0, 1])  # Example true labels

accuracy = accuracy_score(true_labels.numpy(), predictions.numpy())
print(f"Accuracy: {accuracy * 100:.2f}%")

# Close the socket connection
client_socket.close()
server_socket.close()
print("Connection closed.")

# Print the model structure
print(modelpart2)


C:\Users\Exam\.conda\envs\Project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Server listening on 0.0.0.0:10300...
Connected to client at ('172.16.16.77', 58334)
Intermediate output received from client.
Intermediate output shape: torch.Size([5, 13, 128])
Accuracy: 80.00%
Connection closed.
ModelPart2(
  (layers): ModuleList(
    (0): BertLayer(
      (attention): BertAttention(
        (self): BertSdpaSelfAttention(
          (query): Linear(in_features=128, out_features=128, bias=True)
          (key): Linear(in_features=128, out_features=128, bias=True)
          (value): Linear(in_features=128, out_features=128, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (output): BertSelfOutput(
          (dense): Linear(in_features=128, out_features=128, bias=True)
          (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (intermediate): BertIntermediate(
        (dense): Linear(in_features=128, out_features=512, bias=True)
        (intermediat

In [3]:
# Server Side Code (modelpart2)
import socket
import torch
import pickle
from transformers import BertForSequenceClassification
import torch.nn as nn
from sklearn.metrics import accuracy_score

# Server setup
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host = '0.0.0.0'
port = 10300
server_socket.bind((host, port))
server_socket.listen(1)
print(f"Server listening on {host}:{port}...")

# Load TinyBERT model
tinybert = BertForSequenceClassification.from_pretrained("./PSO+Pruning_model/")

# Define modelpart2 to include remaining encoder layers, pooler, and classifier
modelpart2_layers = nn.ModuleList([
    *tinybert.bert.encoder.layer[1:],  # Remaining encoder layers after the first
    tinybert.bert.pooler                # Pooling layer
])

# Combine modelpart2_layers and classifier into a single module
class ModelPart2(nn.Module):
    def __init__(self, layers, classifier):
        super(ModelPart2, self).__init__()
        self.layers = layers
        self.classifier = classifier

    def forward(self, hidden_states):
        # Pass through each encoder layer in modelpart2
        for layer in self.layers[:-1]:  # Exclude the pooling layer for now
            hidden_states = layer(hidden_states)[0]
       
        # Now, use the last layer's output for pooling
        pooled_output = self.layers[-1](hidden_states)  # Pooling layer output
       
        # Classification
        output = self.classifier(pooled_output)  # Classification layer output
        return output

# Instantiate modelpart2
modelpart2 = ModelPart2(modelpart2_layers, tinybert.classifier)

# Accept connection from client
client_socket, client_address = server_socket.accept()
print(f"Connected to client at {client_address}")

# Receive the intermediate tensor from client
data = b''
while True:
    packet = client_socket.recv(4096)
    if not packet:
        break
    data += packet

print("Intermediate output received from client.")

# Deserialize the received data
intermediate_output = pickle.loads(data)
#print(intermediate_output)
# Check the shape of intermediate_output
print(f"Intermediate output shape: {intermediate_output.shape}")  # Add this line for debugging

# Forward pass through modelpart2
with torch.no_grad():
    hidden_states = intermediate_output  # Start from the intermediate output received
    output = modelpart2(hidden_states)  # Get the final output

# Get predictions and calculate accuracy (using example true labels)
predictions = torch.argmax(output ,dim=1)  # Predicted class indices
#true_labels = torch.tensor([1, 0, 1, 0, 1])  # Example true labels
print(predictions)
#print(true_labels)
#accuracy = accuracy_score(true_labels.numpy(), predictions.numpy())
#print(f"Accuracy: {accuracy * 100:.2f}%")

# Close the socket connection
client_socket.close()
server_socket.close()
print("Connection closed.")

Server listening on 0.0.0.0:10300...
Connected to client at ('172.16.16.77', 58863)
Intermediate output received from client.
Intermediate output shape: torch.Size([3, 7, 128])
tensor([1, 1, 1])
Connection closed.


In [2]:
# Server Side Code (modelpart2)
import socket
import torch
import pickle
from transformers import BertForSequenceClassification
import torch.nn as nn
from sklearn.metrics import accuracy_score

# Server setup
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host = '0.0.0.0'
port = 10300
server_socket.bind((host, port))
server_socket.listen(1)
print(f"Server listening on {host}:{port}...")

# Load TinyBERT model
tinybert = BertForSequenceClassification.from_pretrained("./PSO+Pruning_model/")

# Define modelpart2 to include remaining encoder layers, pooler, and classifier
modelpart2_layers = nn.ModuleList([
    *tinybert.bert.encoder.layer[1:],  # Remaining encoder layers after the first
    tinybert.bert.pooler                # Pooling layer
])

# Combine modelpart2_layers and classifier into a single module
class ModelPart2(nn.Module):
    def __init__(self, layers, classifier):
        super(ModelPart2, self).__init__()
        self.layers = layers
        self.classifier = classifier

    def forward(self, hidden_states):
        # Pass through each encoder layer in modelpart2
        for layer in self.layers[:-1]:  # Exclude the pooling layer for now
            hidden_states = layer(hidden_states)[0]
       
        # Now, use the last layer's output for pooling
        pooled_output = self.layers[-1](hidden_states)  # Pooling layer output
       
        # Classification
        output = self.classifier(pooled_output)  # Classification layer output
        return output

# Instantiate modelpart2
modelpart2 = ModelPart2(modelpart2_layers, tinybert.classifier)

# Accept connection from client
client_socket, client_address = server_socket.accept()
print(f"Connected to client at {client_address}")

# Receive the intermediate tensor from client
data = b''
while True:
    packet = client_socket.recv(4096)
    if not packet:
        break
    data += packet

print("Intermediate output received from client.")

# Deserialize the received data
intermediate_output = pickle.loads(data)
#print(intermediate_output)
# Check the shape of intermediate_output
print(f"Intermediate output shape: {intermediate_output.shape}")  # Add this line for debugging

# Forward pass through modelpart2
with torch.no_grad():
    hidden_states = intermediate_output  # Start from the intermediate output received
    output = modelpart2(hidden_states)  # Get the final output

# Get predictions and calculate accuracy (using example true labels)
predictions = torch.argmax(output ,dim=1)  # Predicted class indices
#true_labels = torch.tensor([1, 0, 1, 0, 1])  # Example true labels
print(predictions)
#print(true_labels)
#accuracy = accuracy_score(true_labels.numpy(), predictions.numpy())
#print(f"Accuracy: {accuracy * 100:.2f}%")

# Close the socket connection
client_socket.close()
server_socket.close()
print("Connection closed.")

Server listening on 0.0.0.0:10300...
Connected to client at ('192.168.76.90', 50199)
Intermediate output received from client.
Intermediate output shape: torch.Size([2, 7, 128])
tensor([0, 0])
Connection closed.


In [7]:
modelpart2

ModelPart2(
  (layers): ModuleList(
    (0): BertLayer(
      (attention): BertAttention(
        (self): BertSdpaSelfAttention(
          (query): Linear(in_features=128, out_features=128, bias=True)
          (key): Linear(in_features=128, out_features=128, bias=True)
          (value): Linear(in_features=128, out_features=128, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (output): BertSelfOutput(
          (dense): Linear(in_features=128, out_features=128, bias=True)
          (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (intermediate): BertIntermediate(
        (dense): Linear(in_features=128, out_features=512, bias=True)
        (intermediate_act_fn): GELUActivation()
      )
      (output): BertOutput(
        (dense): Linear(in_features=512, out_features=128, bias=True)
        (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
     

In [9]:
def model_size_in_MB(model):
    total_params = sum(p.numel() for p in model.parameters())
    total_size_in_bytes = total_params * 4  # Assuming float32 (4 bytes)
    total_size_in_MB = total_size_in_bytes / (1024 ** 2)  # Convert bytes to MB
    return total_size_in_MB

# Calculate the model size
size_MB = model_size_in_MB(modelpart2)
print(f"Model size: {size_MB:.2f} MB")


Model size: 0.82 MB


In [10]:
def model_size_in_MB(model):
    total_params = sum(p.numel() for p in model.parameters())
    total_size_in_bytes = total_params * 4  # Assuming float32 (4 bytes)
    total_size_in_MB = total_size_in_bytes / (1024 ** 2)  # Convert bytes to MB
    return total_size_in_MB

# Calculate the model size
size_MB = model_size_in_MB(tinybert)
print(f"Model size: {size_MB:.2f} MB")


Model size: 16.73 MB


In [2]:
# Server Side Code (modelpart2)
import socket
import torch
import pickle
from transformers import BertForSequenceClassification
import torch.nn as nn
from sklearn.metrics import accuracy_score

# Server setup
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host = '0.0.0.0'
port = 10300
server_socket.bind((host, port))
server_socket.listen(1)
print(f"Server listening on {host}:{port}...")

# Load TinyBERT model
tinybert = BertForSequenceClassification.from_pretrained("./PSO+Pruning_model/")

# Define modelpart2 to include remaining encoder layers, pooler, and classifier
modelpart2_layers = nn.ModuleList([
    *tinybert.bert.encoder.layer[1:],  # Remaining encoder layers after the first
    tinybert.bert.pooler                # Pooling layer
])

# Combine modelpart2_layers and classifier into a single module
class ModelPart2(nn.Module):
    def __init__(self, layers, classifier):
        super(ModelPart2, self).__init__()
        self.layers = layers
        self.classifier = classifier

    def forward(self, hidden_states):
        # Pass through each encoder layer in modelpart2
        for layer in self.layers[:-1]:  # Exclude the pooling layer for now
            hidden_states = layer(hidden_states)[0]
       
        # Now, use the last layer's output for pooling
        pooled_output = self.layers[-1](hidden_states)  # Pooling layer output
       
        # Classification
        output = self.classifier(pooled_output)  # Classification layer output
        return output

# Instantiate modelpart2
modelpart2 = ModelPart2(modelpart2_layers, tinybert.classifier)

# Accept connection from client
client_socket, client_address = server_socket.accept()
print(f"Connected to client at {client_address}")

# Receive the intermediate tensor from client
data = b''
while True:
    packet = client_socket.recv(4096)
    if not packet:
        break
    data += packet

print("Intermediate output received from client.")

# Deserialize the received data
intermediate_output = pickle.loads(data)
print(f"Intermediate output shape: {intermediate_output.shape}")  # Debugging line

# Forward pass through modelpart2
with torch.no_grad():
    hidden_states = intermediate_output  # Start from the intermediate output received
    output = modelpart2(hidden_states)  # Get the final output

# Get predictions
predictions = torch.argmax(output, dim=1)  # Predicted class indices
print(f"Predictions: {predictions}")

# Serialize predictions
serialized_predictions = pickle.dumps(predictions)

# Send the predictions back to the client
client_socket.sendall(serialized_predictions)
print("Predictions sent back to client.")

# Close the socket connection
client_socket.close()
server_socket.close()
print("Connection closed.")

Server listening on 0.0.0.0:10300...
Connected to client at ('192.168.76.90', 50352)
Intermediate output received from client.
Intermediate output shape: torch.Size([1, 7, 128])
Predictions: tensor([1])
Predictions sent back to client.
Connection closed.


In [ ]:
# Server Side Code (modelpart2)
import socket
import torch
import pickle
from transformers import BertForSequenceClassification
import torch.nn as nn

# Server setup
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host = '0.0.0.0'
port = 10300
server_socket.bind((host, port))
server_socket.listen(1)
print(f"Server listening on {host}:{port}...")

# Load TinyBERT model
tinybert = BertForSequenceClassification.from_pretrained("./PSO+Pruning_model/")

# Define modelpart2 to include remaining encoder layers, pooler, and classifier
modelpart2_layers = nn.ModuleList([
    *tinybert.bert.encoder.layer[1:],  # Remaining encoder layers after the first
    tinybert.bert.pooler                # Pooling layer
])

# Combine modelpart2_layers and classifier into a single module
class ModelPart2(nn.Module):
    def __init__(self, layers, classifier):
        super(ModelPart2, self).__init__()
        self.layers = layers
        self.classifier = classifier

    def forward(self, hidden_states):
        # Pass through each encoder layer in modelpart2
        for layer in self.layers[:-1]:  # Exclude the pooling layer for now
            hidden_states = layer(hidden_states)[0]
       
        # Now, use the last layer's output for pooling
        pooled_output = self.layers[-1](hidden_states)  # Pooling layer output
       
        # Classification
        output = self.classifier(pooled_output)  # Classification layer output
        return output

# Instantiate modelpart2
modelpart2 = ModelPart2(modelpart2_layers, tinybert.classifier)

while True:
    # Accept connection from client
    client_socket, client_address = server_socket.accept()
    print(f"Connected to client at {client_address}")

    try:
        # Receive the intermediate tensor from client
        data = b''
        while True:
            packet = client_socket.recv(4096)
            if not packet:
                break
            data += packet

        if data:
            print("Intermediate output received from client.")
            # Deserialize the received data
            intermediate_output = pickle.loads(data)
            print(f"Intermediate output shape: {intermediate_output.shape}")

            # Forward pass through modelpart2
            with torch.no_grad():
                hidden_states = intermediate_output  # Start from the intermediate output received
                output = modelpart2(hidden_states)  # Get the final output

            # Get predictions
            predictions = torch.argmax(output, dim=1)  # Predicted class indices
            print(predictions)
            # Serialize and send predictions back to client
            serialized_predictions = pickle.dumps(predictions)
            client_socket.sendall(serialized_predictions)
            print("Predictions sent back to client.")

    except Exception as e:
        print(f"Error occurred: {e}")

    finally:
        # Close the socket connection for the client
        client_socket.close()
        print("Connection closed.")

Server listening on 0.0.0.0:10300...
Connected to client at ('192.168.76.90', 50616)
Intermediate output received from client.
Intermediate output shape: torch.Size([1, 6, 128])
tensor([0])
Predictions sent back to client.
Connection closed.
